In [1]:
from datasets import load_dataset

In [15]:
cats = [
    "Wireless_v1_00",
    "Watches_v1_00",
    "Video_Games_v1_00",
    "Video_DVD_v1_00",
    "Video_v1_00",
    "Toys_v1_00",
    "Tools_v1_00",
    "Sports_v1_00",
    "Software_v1_00",
    "Shoes_v1_00",
    "Pet_Products_v1_00",
    "Personal_Care_Appliances_v1_00",
    "PC_v1_00",
    "Outdoors_v1_00",
    "Office_Products_v1_00",
    "Musical_Instruments_v1_00",
    "Music_v1_00",
    "Mobile_Electronics_v1_00",
    "Mobile_Apps_v1_00",
    "Major_Appliances_v1_00",
    "Luggage_v1_00",
    "Lawn_and_Garden_v1_00",
    "Kitchen_v1_00",
    "Jewelry_v1_00",
    "Home_Improvement_v1_00",
    "Home_Entertainment_v1_00",
    "Home_v1_00",
    "Health_Personal_Care_v1_00",
    "Grocery_v1_00",
    "Gift_Card_v1_00",
    "Furniture_v1_00",
    "Electronics_v1_00",
    "Digital_Video_Games_v1_00",
    "Digital_Video_Download_v1_00",
    "Digital_Software_v1_00",
    "Digital_Music_Purchase_v1_00",
    "Digital_Ebook_Purchase_v1_00",
    "Camera_v1_00",
    "Beauty_v1_00",
    "Baby_v1_00",
    "Automotive_v1_00",
    "Apparel_v1_00",
    "Digital_Ebook_Purchase_v1_01",
    "Books_v1_00",
    "Books_v1_01",
    "Books_v1_02",
]


In [16]:
sorted(cats)

['Apparel_v1_00',
 'Automotive_v1_00',
 'Baby_v1_00',
 'Beauty_v1_00',
 'Books_v1_00',
 'Books_v1_01',
 'Books_v1_02',
 'Camera_v1_00',
 'Digital_Ebook_Purchase_v1_00',
 'Digital_Ebook_Purchase_v1_01',
 'Digital_Music_Purchase_v1_00',
 'Digital_Software_v1_00',
 'Digital_Video_Download_v1_00',
 'Digital_Video_Games_v1_00',
 'Electronics_v1_00',
 'Furniture_v1_00',
 'Gift_Card_v1_00',
 'Grocery_v1_00',
 'Health_Personal_Care_v1_00',
 'Home_Entertainment_v1_00',
 'Home_Improvement_v1_00',
 'Home_v1_00',
 'Jewelry_v1_00',
 'Kitchen_v1_00',
 'Lawn_and_Garden_v1_00',
 'Luggage_v1_00',
 'Major_Appliances_v1_00',
 'Mobile_Apps_v1_00',
 'Mobile_Electronics_v1_00',
 'Music_v1_00',
 'Musical_Instruments_v1_00',
 'Office_Products_v1_00',
 'Outdoors_v1_00',
 'PC_v1_00',
 'Personal_Care_Appliances_v1_00',
 'Pet_Products_v1_00',
 'Shoes_v1_00',
 'Software_v1_00',
 'Sports_v1_00',
 'Tools_v1_00',
 'Toys_v1_00',
 'Video_DVD_v1_00',
 'Video_Games_v1_00',
 'Video_v1_00',
 'Watches_v1_00',
 'Wireless_v1_

In [14]:
ds = load_dataset("amazon_us_reviews")

ValueError: Config name is missing.
Please pick one among the available configs: ['Wireless_v1_00', 'Watches_v1_00', 'Video_Games_v1_00', 'Video_DVD_v1_00', 'Video_v1_00', 'Toys_v1_00', 'Tools_v1_00', 'Sports_v1_00', 'Software_v1_00', 'Shoes_v1_00', 'Pet_Products_v1_00', 'Personal_Care_Appliances_v1_00', 'PC_v1_00', 'Outdoors_v1_00', 'Office_Products_v1_00', 'Musical_Instruments_v1_00', 'Music_v1_00', 'Mobile_Electronics_v1_00', 'Mobile_Apps_v1_00', 'Major_Appliances_v1_00', 'Luggage_v1_00', 'Lawn_and_Garden_v1_00', 'Kitchen_v1_00', 'Jewelry_v1_00', 'Home_Improvement_v1_00', 'Home_Entertainment_v1_00', 'Home_v1_00', 'Health_Personal_Care_v1_00', 'Grocery_v1_00', 'Gift_Card_v1_00', 'Furniture_v1_00', 'Electronics_v1_00', 'Digital_Video_Games_v1_00', 'Digital_Video_Download_v1_00', 'Digital_Software_v1_00', 'Digital_Music_Purchase_v1_00', 'Digital_Ebook_Purchase_v1_00', 'Camera_v1_00', 'Books_v1_00', 'Beauty_v1_00', 'Baby_v1_00', 'Automotive_v1_00', 'Apparel_v1_00', 'Digital_Ebook_Purcha

In [2]:
ds = load_dataset("amazon_reviews_multi", "en")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /home/pl487/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
df = ds["train"].to_pandas()

In [13]:
df["product_category"].value_counts()

product_category
home                        17679
apparel                     15951
wireless                    15717
other                       13418
beauty                      12091
drugstore                   11730
kitchen                     10382
toy                          8745
sports                       8277
automotive                   7506
lawn_and_garden              7327
home_improvement             7136
pet_products                 7082
digital_ebook_purchase       6749
pc                           6401
electronics                  6186
office_product               5521
shoes                        5197
grocery                      4730
book                         3756
baby_product                 3150
furniture                    2984
jewelry                      2747
camera                       2139
industrial_supplies          1994
digital_video_download       1364
luggage                      1328
musical_instruments          1102
video_games                   7

In [11]:
tdf = ds["test"].to_pandas()

In [12]:
tdf["product_category"].value_counts()

product_category
home                        440
apparel                     390
wireless                    374
other                       324
beauty                      301
drugstore                   289
kitchen                     254
toy                         241
sports                      217
automotive                  187
home_improvement            183
digital_ebook_purchase      173
lawn_and_garden             173
pet_products                168
pc                          155
electronics                 153
office_product              139
shoes                       129
grocery                     115
book                        105
jewelry                      89
baby_product                 77
industrial_supplies          64
furniture                    62
camera                       54
digital_video_download       38
luggage                      36
video_games                  30
musical_instruments          25
watch                        13
personal_care_appliance